In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

trade = pd.read_csv('trade_train.csv', index_col=0)
stock = pd.read_csv('stocks.csv'
, index_col=0)
answer = pd.read_csv('answer_sheet.csv')

In [3]:
from DataSet import makeDataset, makeCV, encoding, makeSub
from myModel import linear
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [4]:
seq2seq_df = pd.read_csv('seq2seq_df.csv')

In [5]:
df_rolling23_diff12_MinMax = pd.read_csv('df_rolling23_diff12_MinMax.csv')
#df_rolling23_diff12_MinMax = makeDataset(seq2seq_df, rolling_range=[2,3], diff_range=[1,2], save_na=False, use_minmax=True)
# df_rolling23_diff1_NoMinMax = makeDataset(seq2seq_df, rolling_range=[2,3], diff_range=[1], save_na=False, use_minmax=False)
# df_rolling2_diff12_NoMinmax = makeDataset(seq2seq_df, rolling_range=[2], diff_range=[1,2], save_na=False, use_minmax=False)
# df_rolling2_diff1_NoMinmax = makeDataset(seq2seq_df, rolling_range=[2], diff_range=[1], save_na=False, use_minmax=True)

In [6]:
df_rolling23_diff12_MinMax.head(3)

,종목번호,종목고가_mean,종목고가_max,종목고가_min,종목고가_std,종목저가_mean,종목저가_max,종목저가_min,종목저가_std,기준년월,...,매수고객수rolling_max3,매수고객수rolling_min2,매수고객수rolling_min3,매도고객수diff1,매도고객수diff2,매수고객수diff1,매수고객수diff2,group_char1,group_char4,target
0,A000100,229552.63,237000,219000,4287.89,224578.95,230000,215000,4369.28,201909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0
1,A000100,229552.63,237000,219000,4287.89,224578.95,230000,215000,4369.28,201909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.0
2,A000100,229552.63,237000,219000,4287.89,224578.95,230000,215000,4369.28,201909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0


In [12]:
 X_train, y_train, X_val, y_val, X_test = \
     makeCV(df_rolling23_diff12_MinMax, train_=[201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004])

weight_idx : [0, 6144, 12336, 18528, 24864, 31248, 37680, 44112]


In [13]:

 category_cols = ['종목번호', '시장구분', '표준산업구분코드_대분류', '표준산업구분코드_중분류', '그룹번호']
 X_train_en, X_val_en, X_test_en = encoding(X_train.copy(), y_train.copy(), X_val.copy(), X_test.copy(), category_cols=category_cols, year2020=True, dropTargetZero=True)

train len: 5108


In [14]:
best_feature = [
       '종목번호', '거래량_mean_weekdiff41',  '거래금액_mean', 
       '거래량4_mean', '그룹번호', '매수고객수', '매수고객수rolling_mean2', '매수고객수rolling_mean3', '매수고객수rolling_std2', '매수고객수rolling_std3', '매수고객수rolling_max2', '매수고객수rolling_max3', '매수고객수rolling_min2', '매수고객수rolling_min3', '매수고객수diff1', '매수고객수diff2']

X_train_en = X_train_en[best_feature]
X_val_en = X_val_en[best_feature]
X_test_en = X_test_en[best_feature]

# 48개의 linear model

In [15]:
def lassoModel(train_cols):
    groups = X_train['그룹번호'].unique()
    X_train_en['pred_train']=np.nan
    X_val_en['pred_val']=np.nan
    X_test_en['pred_test']=np.nan
    val_error = []
    coef = np.zeros([48,len(train_cols)])

    for idx, group in enumerate(groups):

        linear_train = X_train_en.loc[X_train['그룹번호']==group, train_cols]
        linear_val = X_val_en.loc[X_val['그룹번호']==group, train_cols]
        linear_test = X_test_en.loc[X_test['그룹번호']==group, train_cols]

        lasso = Lasso(max_iter=2000)
        lasso.fit(linear_train, y_train.iloc[linear_train.index])
        
        X_train_en.loc[X_train['그룹번호']==group, 'pred_train'] = lasso.predict(linear_train)
        X_val_en.loc[X_val['그룹번호']==group, 'pred_val'] = lasso.predict(linear_val)
        X_test_en.loc[X_test['그룹번호']==group, 'pred_test'] = lasso.predict(linear_test)
        
        coef[idx] = lasso.coef_

        mse = mean_squared_error( y_val.iloc[linear_val.index], X_val_en.loc[X_val['그룹번호']==group, 'pred_val'])
        print('lasso validation rmse', group, np.sqrt(mse))
        val_error.append(mse)
    print('[coef == 0]')
    for i, t in enumerate((coef.mean(axis=0) == 0).tolist()):
        if t:
            print(train_cols[i])

    return coef, val_error

In [16]:
coef1, val_error1 = lassoModel(best_feature)

lasso validation rmse MAD01 10.32213854703865
lasso validation rmse MAD02 2.915709713247529
lasso validation rmse MAD03 8.167757218917579
lasso validation rmse MAD04 19.44189876379759
lasso validation rmse MAD05 22.19843266247098
lasso validation rmse MAD06 53.798775989032244
lasso validation rmse MAD07 3.337932151014475
lasso validation rmse MAD08 7.600121797606808
lasso validation rmse MAD09 15.670871947670756
lasso validation rmse MAD10 5.324039427550753
lasso validation rmse MAD11 5.9217879289107955
lasso validation rmse MAD12 5.16625774823753
lasso validation rmse MAD13 10.250416674555606
lasso validation rmse MAD14 8.784008418810734
lasso validation rmse MAD15 4.248895556186751
lasso validation rmse MAD16 3.2758729040642116
lasso validation rmse MAD17 10.351340336879462
lasso validation rmse MAD18 12.986841809081948
lasso validation rmse MAD19 4.282695954323562
lasso validation rmse MAD20 3.773434240867231
lasso validation rmse MAD21 17.060959118650846
lasso validation rmse MAD22

- 10이상 : 01, 04, 09, 13, 14, 17, 18, 21, 38, 42, 44
- 20이상 : 05, 06, 23, 35, 41, 43, 46
- 나머지
